In [ ]:
import cv2
import os
import shutil
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from PIL import Image, ImageOps

In [ ]:
# Get hit number of the video
def get_vid_number(video):
    title = video.split('.')[0].split('Hit_')[-1]
    number = int(title)
    return number

In [ ]:
# Extract the frames we want for each video; we want last 7 frames as this shows ball travelling closer to plate
def extract_frames(video, hit=True):
    title = video.split('.')[0]
    file_name = f'{title}.MOV'
    number = get_vid_number(video)

    vid = cv2.VideoCapture(file_name)
    curr_frame = 1

    for i in range(7):
        success, frame = vid.read()

        if hit:
            cv2.imwrite(f'Hit_{number}_Frame_{curr_frame}.png', frame)
        else:
            cv2.imwrite(f'No_Hit_{number}_Frame_{curr_frame}.png', frame)

        curr_frame += 1

    print(f'Finished: {video}')

    os.remove(video)

In [ ]:
# Define the custom transformation function with centered cropping and smaller size
def custom_transform(image, crop_size=(650, 650)):
    width, height = image.size  # Get the width and height of the original image

    crop_width, crop_height = crop_size  # Custom size for cropping

    # Calculate the center of the image
    center_x, center_y = width // 2, height // 2

    # Calculate the cropping box coordinates
    left = center_x - crop_width // 2
    upper = center_y - crop_height // 2
    right = center_x + crop_width // 2
    lower = center_y + crop_height // 2

    # Ensure the crop stays within the image boundaries
    if left < 0:
        left = 0
        right = crop_width
    if right > width:
        right = width
        left = width - crop_width
    if upper < 0:
        upper = 0
        lower = crop_height
    if lower > height:
        lower = height
        upper = height - crop_height

    # Crop the image centered around the middle area
    image = image.crop((left, upper, right, lower))

    # Convert the image to an OpenCV format (BGR)
    bgr_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # Define the color range for the ball (example values)
    lower_color = (135, 135, 135)  # Lower bound for ball color
    upper_color = (255, 255, 255)  # Upper bound for ball color

    # Create a mask to identify the ball
    mask = cv2.inRange(bgr_image, lower_color, upper_color)

    # Apply the mask to the original image
    masked_image = cv2.bitwise_and(bgr_image, bgr_image, mask=mask)

    # Convert the masked image back to RGB format
    masked_image = cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB)

    # Convert the image to greyscale
    image = Image.fromarray(masked_image)
    image = ImageOps.grayscale(image)

    # Resize the image to 128x128 pixels
    image = image.resize((128, 128))

    # Convert to a tensor
    image_tensor = transforms.ToTensor()(image)

    return image_tensor

In [ ]:
# Start with the Hit Folder
def prepare_videos(swing_decision):
  # Change directory to folder of videos files downloaded
  home = Path.home()
  frames_path = os.path.join(home, 'Downloads', 'Wii_Baseball_CNN_RNN', f'{swing_decision}')

  os.chdir(frames_path)

  # Loop through every video and feed through extract_frames function
  num_videos = len(os.listdir())
  for video in range(num_videos):
      if video_list[video] != '.DS_Store':
          extract_frames(video_list[video], hit=True)

  print("Finished converting all video files to their frames\n")

  # Create list of all file paths of the frames
  frame_files = os.listdir()
  frame_files = [os.path.join(frames_path, f) for f in frame_files if f.endswith('.png')]

  # Apply
  for frame in frame_files:
    opened_frame = Image.open(frame)
    frame_tensor = custom_transform(opened_frame)
    image = transforms.ToPILImage()(frame_tensor)
    frame_path = os.path.expanduser(frame)
    file_name, _ = os.path.splitext(os.path.basename(frame_path))[0]
    new_name = f'{file_name}_FINAL.png'
    image.save(new_name)
    os.remove(frame)

  print("Finished preparing all frames")

prepare_videos('Hit')
prepare_videos('No_Hit')

In [ ]:
def combine_folders(hit_folder, no_hit_folder, prepared_frames):
  Path(prepared_frames).mkdir(exist_ok = True)

  for file_name in os.listdir(hit_folder):
    if file_name not file_name.endswith('.png'):
      continue
    source_file = os.path.join(hit_folder, file_name)
    shutil.copy(source_file, prepared_frames)

  for file_name in os.listdir(no_hit_folder):
    if file_name not file_name.endswith('.png'):
      continue
    source_file = os.path.join(no_hit_folder, file_name)
    shutil.copy(source_file, prepared_frames)

  print(f"All files combined into: {prepared_frames}")

home = Path.home()

hit_folder = os.path.join(home, 'Downloads', 'Wii_Baseball_CNN_RNN', 'Hit')
no_hit_folder = os.path.join(home, 'Downloads', 'Wii_Baseball_CNN_RNN', 'No_Hit')
prepared_frames = os.path.join(home, 'Downloads', 'Wii_Baseball_CNN_RNN', 'Prepared_Frames')

combine_folders(hit_folder, no_hit_folder, prepared_frames)